<a href="https://colab.research.google.com/github/pvidya11/Deep_Learning/blob/main/Projects/01_MNIST_Fashion_Classification/CNN_MNIST_Fashion_Optimize_using_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
print(tf.__version__)

2.4.1


In [3]:
fashion_mnist = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
(x_train, y_train),(x_test,y_test) = fashion_mnist

In [5]:
(x_train.shape, y_train.shape),(x_test.shape,y_test.shape)

(((60000, 28, 28), (60000,)), ((10000, 28, 28), (10000,)))

In [6]:
#normalization
x_train = x_train/255
x_test  = x_test/255

In [7]:
x_train[0].shape

(28, 28)

In [8]:
x_train_img = x_train.reshape(len(x_train),28,28,1)
x_test_mg = x_test.reshape(len(x_test),28,28,1)

In [9]:
!pip install -U keras-tuner

     |████████████████████████████████| 71kB 5.3MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=4bbf67c16284495cd104cbcb6d6b151ac432dbc397bf3905856aa5b93254fb59
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=c93b297d608b5044a1fef387a7df2ad802a5956afbfbe63b208bc44eb1e6076b
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [10]:

from tensorflow import keras
from tensorflow.keras.layers import Conv2D,Dense,Flatten
from kerastuner.tuners import RandomSearch


def build_model(hp):
    model = keras.Sequential()
    model.add(Conv2D(filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                            kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
                            activation = 'relu',
                            input_shape = (28,28,1)
    ))
    model.add(Conv2D(filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
                            kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
                            activation = 'relu'
    ))
    model.add(Flatten())
    model.add(Dense(units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
                            activation='relu'))
    model.add(Dense(units=10,activation='softmax'))

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model



In [11]:
from kerastuner.engine.hyperparameters import HyperParameters

In [12]:
tuner_search = RandomSearch(hypermodel = build_model,
                            objective='val_accuracy',
                            max_trials=5,
                            directory='output',
                            project_name="MNIST_Fashion")

In [13]:
tuner_search.search(x_train_img, y_train, 
                    epochs = 3, 
                    validation_split=0.2
                    )

Trial 5 Complete [00h 00m 15s]
val_accuracy: 0.8722500205039978

Best val_accuracy So Far: 0.8901666402816772
Total elapsed time: 00h 02m 04s
INFO:tensorflow:Oracle triggered exit


In [20]:
best_model = tuner_search.get_best_models(num_models=1)[0]

In [21]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        13856     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                1769568   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 1,784,874
Trainable params: 1,784,874
Non-trainable params: 0
_________________________________________________________________


In [22]:
best_model.fit(x_train_img,y_train,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2775 - accuracy: 0.9005 - val_loss: 0.2823 - val_accuracy: 0.8957
Epoch 5/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.2533 - accuracy: 0.9088 - val_loss: 0.2712 - val_accuracy: 0.9020
Epoch 6/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.2269 - accuracy: 0.9203 - val_loss: 0.2687 - val_accuracy: 0.9043
Epoch 7/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.2061 - accuracy: 0.9260 - val_loss: 0.2766 - val_accuracy: 0.8970
Epoch 8/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1914 - accuracy: 0.9316 - val_loss: 0.2611 - val_accuracy: 0.9088
Epoch 9/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1695 - accuracy: 0.9394 - val_loss: 0.2543 - val_accuracy: 0.9097
Epoch 10/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1550 - accuracy: 0.9449 - val_loss: 0.2514 - val_accuracy